In [1]:
# correlation maps visiualization

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# 定义文件路径
paths = [
    r"C:\Users\12770\Desktop\project_ym\SVCA\Result\m010iso2\400s-900s(awake)\neuron_correlation_statistics",
    r"C:\Users\12770\Desktop\project_ym\SVCA\Result\m010iso2\50s-400s(supp_only)\neuron_correlation_statistics",
    r"C:\Users\12770\Desktop\project_ym\SVCA\Result\m010iso2\50s-400s(burst_only)\neuron_correlation_statistics"
]

# 输出文件夹路径
output_dir = r"C:\Users\12770\Desktop\project_ym\SVCA\Result\m010iso2\neuron_correlation_statistics\comparison_of_burst_supp_awake"

# 确保输出文件夹存在
os.makedirs(output_dir, exist_ok=True)

# 加载数据
all_correlations = []
top_1000_correlations = []
for path in paths:
    with open(os.path.join(path, "variables_Test_Onlyl0k0.pkl"), 'rb') as f:
        data = pickle.load(f)
        all_correlations.append(data['correlations'])  # 假设这代表所有神经元的相关数组
        top_1000_correlations.append(data['top_1000_correlations'])  # 假设这代表前1000神经元的相关数组

# 计算p值和显著性符号
def calculate_p_values_and_significance(data_list):
    num_conditions = len(data_list)
    p_values = np.zeros((num_conditions, num_conditions))
    significance = np.empty((num_conditions, num_conditions), dtype=object)

    for i in range(num_conditions):
        for j in range(i+1, num_conditions):
            _, p_value = ttest_ind(data_list[i], data_list[j])
            p_values[i, j] = p_value
            if p_value < 0.001:
                significance[i, j] = '***'
            elif p_value < 0.01:
                significance[i, j] = '**'
            elif p_value < 0.05:
                significance[i, j] = '*'
            else:
                significance[i, j] = 'ns'

    return p_values, significance

all_p_values, all_significance = calculate_p_values_and_significance(all_correlations)
top_1000_p_values, top_1000_significance = calculate_p_values_and_significance(top_1000_correlations)

# 绘图函数
def plot_comparison(correlations, p_values, significance, title, labels, filename_base):
    means = [np.mean(corr) for corr in correlations]
    errors = [np.std(corr) / np.sqrt(len(corr)) * 1.96 for corr in correlations]

    plt.figure(figsize=(10, 6))
    x = np.arange(len(means))
    plt.bar(x, means, yerr=errors, capsize=5, alpha=0.7, color=['red', 'blue', 'green'])
    plt.xticks(x, labels)
    plt.ylabel('Average Pearson Correlation')
    plt.title(title)

    # 添加显著性标记
    for i in range(len(significance)):
        for j in range(i + 1, len(significance)):
            y = max(means[i], means[j]) + 0.02
            plt.text((x[i] + x[j]) / 2, y, significance[i, j], ha='center', va='bottom')

    plt.tight_layout()
    
    # 保存为PNG和PDF格式
    plt.savefig(os.path.join(output_dir, f"{filename_base}.png"))
    plt.savefig(os.path.join(output_dir, f"{filename_base}.pdf"))
    plt.close()

    # 输出p值和显著性
    print(f"P-values and significance for {title}:")
    for i in range(len(p_values)):
        for j in range(i + 1, len(p_values)):
            print(f"Comparison: {labels[i]} vs {labels[j]}, p-value: {p_values[i, j]:.4f}, significance: {significance[i, j]}")

# 定义标签和标题
condition_labels = ['400s-900s(awake)', '50s-400s(supp_only)', '50s-400s(burst_only)']

# 生成并保存图像
plot_comparison(all_correlations, all_p_values, all_significance, 'All Neurons Comparison', condition_labels, 'all_neurons_comparison')
plot_comparison(top_1000_correlations, top_1000_p_values, top_1000_significance, 'Top 1000 Neurons Comparison', condition_labels, 'top_1000_neurons_comparison')


P-values and significance for All Neurons Comparison:
Comparison: 400s-900s(awake) vs 50s-400s(supp_only), p-value: 0.0000, significance: ***
Comparison: 400s-900s(awake) vs 50s-400s(burst_only), p-value: 0.0000, significance: ***
Comparison: 50s-400s(supp_only) vs 50s-400s(burst_only), p-value: 0.0000, significance: ***
P-values and significance for Top 1000 Neurons Comparison:
Comparison: 400s-900s(awake) vs 50s-400s(supp_only), p-value: 0.0000, significance: ***
Comparison: 400s-900s(awake) vs 50s-400s(burst_only), p-value: 0.0000, significance: ***
Comparison: 50s-400s(supp_only) vs 50s-400s(burst_only), p-value: 0.0000, significance: ***


In [2]:
# correlation maps visiualization
###————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————###
# 数据和结果路径配置
base_load_path = 'C:/Users/12770/Desktop/project_ym/Data/'
base_res_path = 'C:/Users/12770/Desktop/project_ym/SVCA/Result/'

mice_ids = ['m005iso1', 'm009iso1', 'm009iso2', 'm010iso1', 'm010iso2']
mice_ids = ['m010iso2']
kbd_evt_values = {
    'm010iso2': {'KBD1': 191.4293, 'EVT06': 10.41715},
    'm010iso1': {'KBD1': 235.23225, 'EVT06': 34.06705},
    'm009iso2': {'KBD1': 158.6044, 'EVT06': 36.249525},
    'm009iso1': {'KBD1': 146.826075, 'EVT06': 18.017625},
    'm005iso1': {'KBD1': 176.819025, 'EVT06': 34.28035}
}
###————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————###

from scaling_analysis.utils_ym import plot_neuron_correlation

def plot_neuron_correlation(real_activity, predicted_activity, centers, res_path, suffix):
    correlations = []
    for i in range(real_activity.shape[1]):
        if np.std(real_activity[:, i]) == 0 or np.std(predicted_activity[:, i]) == 0:
            correlations.append(0)
        else:
            corr = np.corrcoef(real_activity[:, i], predicted_activity[:, i])[0, 1]
            correlations.append(corr)
    correlations = np.nan_to_num(correlations, nan=0) 
    correlations = np.maximum(correlations, 0)
    
    s = max(2, 300 / np.sqrt(centers.shape[1]))  

    np.save(os.path.join(res_path, 'real_predicted_activity_centers.npy'), {
            'real_activity': real_activity,
            'predicted_activity': predicted_activity,
            'centers': centers,
        })

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(
        centers[0, :], 
        centers[1, :], 
        c=correlations,
        cmap='Reds', 
        s=10, 
        alpha=correlations,
        edgecolors='none',
        vmin=0, 
        vmax=1
    )

    cbar = plt.colorbar(scatter)
    cbar.set_label('Pearson Correlation')
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'Neuron Activity Correlation ({suffix})')
    
    save_dir = os.path.join(res_path, "neuron_correlation_maps")
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, f"neuron_correlation_{suffix}.png"), dpi=300)
    # plt.savefig(os.path.join(save_dir, f"neuron_correlation_{suffix}.pdf"))
    plt.close()   

for mouse_id in mice_ids:
    load_path = os.path.join(base_load_path, mouse_id)
    res_path = os.path.join(base_res_path, mouse_id)

    print(f"Processing data for {mouse_id}")
    params = np.load(os.path.join(res_path, 'real_predicted_activity_centers_Test_Onlyl0k0.npy'), allow_pickle=True).item()
    real_activity = params['real_activity']
    predicted_activity = params['predicted_activity']
    centers = params['centers']
    plot_neuron_correlation(real_activity, predicted_activity, centers, res_path, "Test")

Processing data for m010iso2


In [ ]:
# correlation maps statistical 2
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import ttest_ind

def load_correlations(filepath):
    data = np.load(filepath, allow_pickle=True).item()
    return data['correlations']

def compute_mean_and_sem(correlations):
    mean_corr = np.mean(correlations)
    sem_corr = np.std(correlations) / np.sqrt(len(correlations))
    return mean_corr, sem_corr

def plot_average_correlation(correlations_ane, correlations_awake, res_path, title_suffix):
    # 计算平均值和SEM
    mean_all_ane, sem_all_ane = compute_mean_and_sem(correlations_ane)
    mean_all_awake, sem_all_awake = compute_mean_and_sem(correlations_awake)

    # 用t检验评估显著性
    t_stat, p_val_all = ttest_ind(correlations_ane, correlations_awake, equal_var=False)
    significance = "***" if p_val_all < 0.001 else ""

    # 绘制柱状图
    labels = ['Ane', 'Awake']
    means = [mean_all_ane, mean_all_awake]
    errors = [1.96 * sem_all_ane, 1.96 * sem_all_awake]

    plt.figure(figsize=(8, 6))
    bars = plt.bar(labels, means, yerr=errors, color=['black', 'black'], alpha=1, capsize=5)
    plt.ylabel('Average Pearson Correlation')
    plt.title(f'Neuron Correlation ({title_suffix})')

    # 添加显著性标记
    max_height = max(means)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height + 0.01 * max_height, significance,
                 ha='center', va='bottom', fontsize=12)

    save_dir = os.path.join(res_path, "correlation_comparison")
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, f"correlation_comparison_{title_suffix.replace(' ', '_')}.png"), dpi=300)
    plt.savefig(os.path.join(save_dir, f"correlation_comparison_{title_suffix.replace(' ', '_')}.pdf"))
    plt.close()

# 使用示例：
res_path_ane = "C:\\Users\\12770\\Desktop\\project_ym\\SVCA\\Result\\m010iso2\\50s-400s(ane)"
res_path_awake = "C:\\Users\\12770\\Desktop\\project_ym\\SVCA\\Result\\m010iso2\\400s-900s(awake)"

# 加载数据
correlations_ane = load_correlations(os.path.join(res_path_ane, "correlations_Test_Only.npy"))
correlations_awake = load_correlations(os.path.join(res_path_awake, "correlations_Test_Only.npy"))

# 所有神经元
plot_average_correlation(correlations_ane, correlations_awake, res_path_ane, "All Neurons")

# 前1000个神经元
sorted_idx_ane = np.argsort(correlations_ane)[::-1][:1000]
sorted_idx_awake = np.argsort(correlations_awake)[::-1][:1000]
top_1000_ane = correlations_ane[sorted_idx_ane]
top_1000_awake = correlations_awake[sorted_idx_awake]
plot_average_correlation(top_1000_ane, top_1000_awake, res_path_ane, "Top 1000 Neurons")
